In [91]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [92]:
# 1. 파일 설정
SEARCH_QUERY_PATH = "search_query_list.csv"
RESULT_CSV_PATH = "naver_blog_reviews.csv"
MAX_POSTS_PER_QUERY = 20
DELAY_BETWEEN_PAGES = 2

In [93]:
# 2. 검색어 불러오기
queries_df = pd.read_csv(SEARCH_QUERY_PATH)
queries = queries_df["search_query"].tolist()

In [94]:
# 3. ChromeDriver 설정
options = Options()
# options.add_argument("--headless")  # GUI 없이 실행하려면 주석 해제
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [95]:
# 4. 결과 저장 리스트
results = []

In [96]:
def crawl_naver_blog(query, max_posts=1):
    base_url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword="
    driver.get(base_url + query)
    time.sleep(DELAY_BETWEEN_PAGES)

    links = driver.find_elements(By.CSS_SELECTOR, "a.desc_inner")

    # print(f"페이지 링크 수: {len(links)}")
    # for l in links:
    #     print(" -", l.text)

    count = 0

    for idx, link in enumerate(links):
        if count >= max_posts:
            break
        try:
            href = link.get_attribute('href')
            title = link.text

            # 새 탭 열고 이동
            driver.execute_script("window.open(arguments[0]);", href)
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(2)

            blog_url = driver.current_url

             # iframe 진입 (네이버 블로그는 대부분 iframe 내에 본문 존재)
            try:
                iframe = driver.find_element(By.CSS_SELECTOR, "iframe#mainFrame")
                driver.switch_to.frame(iframe)
                time.sleep(1)
            except:
                print("iframe 진입 실패 (외부 블로그일 수 있음)")

            # 본문 및 작성일 추출
            content_elements = driver.find_elements(By.CSS_SELECTOR, 'div.se-main-container, div#postViewArea')
            content_text = "\n".join([elem.text for elem in content_elements]).strip()

            date_elements = driver.find_elements(By.CSS_SELECTOR, 'span.se_publishDate, span.date')
            date_text = date_elements[0].text if date_elements else ""

            # 프레임 복귀
            driver.switch_to.default_content()

            # 관광지명 추출
            park_name = query.split()[0]

            # print(f"본문 길이: {len(content_text)}")  # 디버깅용 로그

            if len(content_text) >= 1000:
                results.append({
                    "search_query": query,
                    "title": title,
                    "content": content_text,
                    "date": date_text,
                    "blog_url": blog_url,
                    "park_name": park_name
                })
                count += 1
            # else:
                # print("본문이 너무 짧아 저장되지 않음")

            print(f"상태: {idx+1}/{len(links)}", end=' -> ')

            # 탭 닫고 복귀
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print(f"Error at link {idx}: {e}")
            
            # 열린 탭만 닫고, driver 세션이 유지 중인지 확인
            if len(driver.window_handles) > 1:
                try:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    print("세션이 이미 종료되었거나 유효하지 않음")
                    break  # 또는 return으로 함수 탈출

    print()


In [97]:
# 6. 전체 쿼리 크롤링 실행
# 상위 5개만 테스트
# test_queries = queries[:3]

for i, query in enumerate(queries):
    print(f"{i+1}번째 크롤링: {query}")
    crawl_naver_blog(query, MAX_POSTS_PER_QUERY)

1번째 크롤링: 도산근린공원 후기
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
2번째 크롤링: 도산근린공원 리뷰
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
3번째 크롤링: 도산근린공원 산책
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
4번째 크롤링: 도산근린공원 풍경
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
5번째 크롤링: 도산근린공원 힐링
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
6번째 크롤링: 도산근린공원 단풍
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
7번째 크롤링: 도산근린공원 벚꽃
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
8번째 크롤링: 율현공원 후기
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
9번째 크롤링: 율현공원 리뷰
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
10번째 크롤링: 율현공원 산책
상태: 1/7 -> 상태: 2/7 -> 상태: 3/7 -> 상태: 4/7 -> 상태: 5/7 -> 상태: 6/7 -> 상태: 7/7 -> 
11번째 크롤링: 율현공원 풍경
상태: 1/7 -> 상태: 2/

In [98]:
# 7. 결과 저장
df = pd.DataFrame(results)
df.to_csv(RESULT_CSV_PATH, index=False)

print(len(results))

# 8. 크롬 종료
driver.quit()

1088
